In [3]:
import sys
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
from matplotlib import pylab as plt

import sklearn
sklearn.set_config(enable_metadata_routing=True)

from mapie.metrics import regression_coverage_score, regression_mean_width_score
from mapie.subsample import BlockBootstrap
from mapie.regression import MapieTimeSeriesRegressor, MapieRegressor

import torch
import json
import pickle

from sklearn.metrics import mean_squared_error

In [4]:
sys.path.append(r'C:\Users\obhlivoj\DP\System-Imbalance-Forecasting\models\ML')

from config import get_config
from train import get_ds, get_model, train_model
from transformer_dataset import TSDataset

path = r'C:\Users\obhlivoj\DP\System-Imbalance-Forecasting\models\ML'

In [5]:
from sklearn.base import BaseEstimator, RegressorMixin
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg = get_config()

train_scl, val_scl, test_scl = get_ds(cfg, return_raw=True)
train_ds = TSDataset(
    train_scl, cfg['src_seq_len'], cfg['tgt_seq_len'])
val_ds = TSDataset(val_scl, cfg['src_seq_len'], cfg['tgt_seq_len'])
test_ds = TSDataset(test_scl, cfg['src_seq_len'], cfg['tgt_seq_len'])

# read json_info
with open(f'{path}/final_params.json', 'r') as file:
    best_params = json.load(file)

for param, value in best_params['best_params'][0].items():
    cfg[param] = value

model = get_model(cfg)

In [7]:
cv_mapiets = BlockBootstrap(
    n_resamplings=4, n_blocks=10, overlapping=False, random_state=69
)

alpha = 0.05

In [8]:
x_list = []
y_list = []
for item in train_ds+val_ds:
    x_list.append(item["encoder_input"])
    y_list.append(item["label"].squeeze())

X = torch.stack(x_list).numpy()
y = torch.stack(y_list).numpy()

In [9]:
class PyTorchRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, model, cfg, device):
        self.model = model.to(device)
        self.cfg = cfg
        self.device = device

    def fit(self, X, y):
        # Convert X and y to PyTorch tensors and move them to the specified device
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).to(self.device)
        
        # Prepare dataset
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=self.cfg['batch_size'], shuffle=True)

        # Initialize optimizer and loss function
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.cfg['lr'], eps=1e-9)
        loss_fn = nn.MSELoss().to(self.device)

        # Training loop
        self.model.train()
        for _ in range(self.cfg['num_epochs']):
            for encoder_input, label in dataloader:
                optimizer.zero_grad()

                # Forward pass
                output = self.model(encoder_input)
                loss = loss_fn(output.view(-1), label.view(-1))

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

        return self

    def predict(self, X):
        # Ensure model is in evaluation mode
        self.model.eval()

        # Convert X to a tensor and move to the specified device
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)

        # Make predictions
        with torch.no_grad():
            predictions = self.model(X_tensor)

        # Move predictions to CPU for compatibility with scikit-learn
        return predictions.cpu().numpy().squeeze()

    def score(self, X, y):
        predictions = self.predict(X)
        return mean_squared_error(predictions, y, squared=True)

In [10]:
mlp_mapie = PyTorchRegressor(model, cfg, device)

In [11]:
mapie_ts = MapieTimeSeriesRegressor(
    mlp_mapie, method="enbpi", cv=cv_mapiets, agg_function="mean", n_jobs=-1
)
mapie_ts.fit(X, y)

MapieTimeSeriesRegressor(cv=BlockBootstrap(length=None, n_blocks=10, n_resamplings=4, overlapping=False,
        random_state=69),
                         estimator=PyTorchRegressor(cfg={'batch_size': 64,
                                                         'data_pickle_name': 'merged_data.pkl',
                                                         'diffs': None,
                                                         'dropout': 0.5,
                                                         'exo_vars': ['month_sin',
                                                                      'month_cos',
                                                                      'day_sin',
                                                                      'day_cos',
                                                                      'weekday_sin',
                                                                      'weekday_cos',
                                                                      'hour_sin',
                                                                      'hour_cos',
                                                                      'quarter_hour_...
                                                         'src_seq_len': 32,
                                                         'target': ['system_imbalance'],
                                                         'test_split': '2023-06-01 '
                                                                       '00:00:00',
                                                         'tgt_seq_len': 1,
                                                         'tgt_step': 0,
                                                         'train_split': '2023-03-01 '
                                                                        '00:00:00'},
                                                    device=device(type='cuda'),
                                                    model=MLP(
  (fc1): Linear(in_features=725, out_features=64, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)),
                         n_jobs=-1)

In [12]:
x_list = []
y_list = []
for item in test_ds:
    x_list.append(item["encoder_input"])
    y_list.append(item["label"].squeeze())

X_test = torch.stack(x_list).numpy()
y_test = torch.stack(y_list).numpy()

In [13]:
y_pred, y_pis = mapie_ts.predict(X_test, alpha=alpha)
coverage = regression_coverage_score(y_test, y_pis[:, 0, 0], y_pis[:, 1, 0])
width = regression_mean_width_score(y_pis[:, 0, 0], y_pis[:, 1, 0])

In [14]:
mean_squared_error(mapie_ts.predict(X_test), y_test, squared=False)

108.678055

In [15]:
width

467.4986341264513

In [16]:
mlp_res = {
    "y_pred": y_pred,
    "y_pis": y_pis,
    "coverage": coverage,
    "width": width,
    "y_true": y_test
}

with open('./results/mlp_dict.pkl', 'wb') as f:
    pickle.dump(mlp_res, f)